# Tutorial 3: Unfitted Poisson problem

In this **tutorial**, we will learn

- How to formulate an unfitted finite element method
- How to construct an unfitted boundary geometry setup
- How to use the Aggregated Finite Element Method
- How to impose Dirichlet boundary conditions _weakly_

## Problem statement

We want to find the scalar field $u$ such that

$$
\left\lbrace
\begin{aligned}
-\Delta u &= 0 \ &\text{in} \ \Omega,\\
u &= x - y \ &\text{on}\ \partial\Omega,
\end{aligned}
\right.
$$

where $\Omega$ is the crescent moon 2D shape

<img src="crescent_moon.png" width="280">

## Numerical scheme

We will use unfitted FEM.

The main motivation is to avoid meshing complicated geometries.

The main idea is to embed $\Omega$ into a background grid $\tilde{\Omega}$.

FE spaces are formulated on the grid $\tilde{\Omega}$, while the weak form is integrated in $\Omega$.


| Body-fitted | Unfitted |
| ----  | ---- |
| <img src="fig_2_body-fitted_a_bis.png" width="240"> | <img src="fig_3_unfitted.png" width="240">

In unfitted, we have no control over how mesh cuts geometry
* numerical integration of the weak form is more involved
* Dirichlet boundary conditions must be weakly imposed
* standard FEMs are generally not stable and ill-conditioned

We show next a typical unfitted FE simulation pipeline in [GridapEmbedded](https://github.com/gridap/GridapEmbedded.jl)

We use Aggregated FEM, which is robust to small cut cells

In [15]:
using Gridap
using GridapEmbedded

### Embedded boundary representation

We use the level-set method and constructive solid geometry

$\partial\Omega$ is represented as the zero-level-set of a given $\psi$ by

$$
\partial\Omega = \{ (x,y) \ | \ \psi(x,y) = 0 \}
$$

Note that $\partial\Omega$ is the result of subtracting two disks:

<img src="crescent_moon_setup.png" width="280">

We use this to construct $\psi$

In [16]:
R  = 0.5
L  = 0.5*R
p1 = Point(0.0,0.0)
p2 = p1 + VectorValue(-L,L)

geo1 = disk(R,x0=p1) # Lower right disk
geo2 = disk(R,x0=p2) # Upper left disk
geo3 = setdiff(geo1,geo2)

AnalyticalGeometry(Node((:-, "", nothing),Leaf((GridapEmbedded.LevelSetCutters.var"#diskfun#9"{VectorValue{2, Float64}, Float64}((0.0, 0.0), 0.5), "disk", GridapEmbedded.LevelSetCutters.BoundingBox{2, Float64}((-0.505, -0.505), (0.505, 0.505)))),Leaf((GridapEmbedded.LevelSetCutters.var"#diskfun#9"{VectorValue{2, Float64}, Float64}((-0.25, 0.25), 0.5), "disk", GridapEmbedded.LevelSetCutters.BoundingBox{2, Float64}((-0.755, -0.255), (0.255, 0.755))))))

### Unfitted triangulations

We generate a background grid containing $\partial\Omega$

In [17]:
t = 1.01
pmin = p1-t*R
pmax = p1+t*R

n = 30
partition = (n,n)
bgmodel = CartesianDiscreteModel(pmin,pmax,partition)
dp = pmax - pmin

VectorValue{2, Float64}(1.01, 1.01)

We need "Active" and "Physical" triangulations (in gray below)


| Active | Physical |
| ----  | ---- |
| _For_ FE spaces (FE bases, DoFs) | _For_ integrating the weak form |
| <img src="fig_active_triangulation.png" width="280"> | <img src="fig_physical_trian_1.png" width="280">

To generate them, we cut first the embedded geometry against the model

In [18]:
cutgeo = cut(bgmodel,geo3)

EmbeddedDiscretization()

#### Active triangulation

In [19]:
Ω_act = Triangulation(cutgeo,ACTIVE)

BodyFittedTriangulation()

We plot the active trian (gray) on the background grid

In [20]:
Ω_bg = Triangulation(bgmodel)
writevtk(Ω_bg,"bg_trian")
writevtk(Ω_act,"act_trian")

(["act_trian.vtu"],)

<img src="fig_active_triangulation.png" width="280">

#### Physical triangulation

In [21]:
Ω = Triangulation(cutgeo,PHYSICAL)
writevtk(Ω,"phys_trian")

(["phys_trian.vtu"],)

We plot the physical trian (gray) on the background grid

<img src="fig_physical_trian_1.png" width="280">

Subtriangulations on cut cells are used to restrict integration to $\Omega$

<img src="fig_physical_trian_2.png" width="280">

We also plot the active trian on top of the physical one

<img src="fig_physical_trian_3.png" width="280">

### Unfitted FE spaces

#### AgFEM approximation

The main idea is to remove DoFs on cut cells with very small local support.

To do that, we constrain exterior DoFs ${\color{red}\times}$, in terms of interior DOFs ${\color{blue}\bullet}$, as follows:
1. We map each exterior DoF ${\color{red}\times}$ to an interior cell $\tilde{K}({\color{red}\times})$ of $\Omega_{\rm act}$ (aggregation).
2. We extrapolate the DoF value of ${\color{red}\times}$ with the local FE functions of $\tilde{K}({\color{red}\times})$.

This leads to:
$$
  u_{{\color{red}\times}} = \sum_{{\color{blue}\bullet}\in \tilde{K}({\color{red}\times})} \varphi_{{\color{blue}\bullet}}(x_{{\color{red}\times}})u_{{\color{blue}\bullet}}, \quad \forall {\color{red}\times} \in \mathcal{E}
$$

# <img src="fig_agfemspace.png" width="280">

For an AgFEM space, we generate first a standard linear FE space **on the `ACTIVE` triangulation**.

Note that we do not prescribe any strong Dirichlet BCS.

In [22]:
order = 1
reffe = ReferenceFE(lagrangian,Float64,order)
Vstd = TestFESpace(Ω_act,reffe)

UnconstrainedFESpace()

We construct the aggregates

In [23]:
strategy = AggregateAllCutCells()
aggregates = aggregate(strategy,cutgeo)

900-element Vector{Int32}:
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
 42
 42
 43
  ⋮
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0

The AgFEM spaces are

In [24]:
V = AgFEMSpace(Vstd,aggregates)
U = TrialFESpace(V)

FESpaceWithLinearConstraints()

### Unfitted measures

We define integration measures **on the `PHYSICAL` triangulation**.

In [25]:
degree = 2*order
dΩ = Measure(Ω,degree)

Measure()

In general, we must do `degree = 2*dim*order` because Fubini theorem does not hold on cut regions

See commented tutorial for more details

### Imposing Dirichlet boundary conditions

We must impose Dirichlet BCs _weakly_

| Model problem | Body-fitted | Unfitted |
| ---- | ---- | ---- |
| <img src="fig_cp_bcs.png" width="180"> | <img src="fig_bf_bcs.png" width="180"> | <img src="fig_ud_bcs.png" width="180"> |
| | Remove Dirichlet DoFs | No DoFs on $\Gamma_{\rm D}$!!! |

For this purpose, we use Nitsche's method:
$$
\begin{aligned}
a_K(u,v) &\doteq \int_{\Omega \cap K} \nabla u \cdot \nabla v \ \mathrm{d}\Omega + \int_{\Gamma_{\rm D} \cap K} \beta_K uv -  (\nabla u \cdot \boldsymbol{n} ) v - (\nabla v \cdot \boldsymbol{n} ) u \ \mathrm{d}\Omega \\
l_K(v) &\doteq \int_{\Omega \cap K} f v \ \mathrm{d}\Omega + \int_{\Gamma_{\rm N} \cap K} g v \ \mathrm{d}\Gamma + \int_{\Gamma_{\rm D} \cap K} \beta_K u_{\rm D}v - (\nabla v \cdot \boldsymbol{n} ) u_{\rm D} \ \mathrm{d}\Omega
\end{aligned}
$$
where $\beta_K$ must be large enough to ensure well-posedness.

Thus, we need to integrate boundary terms on the embedded boundary

In [26]:
Γ = EmbeddedBoundary(cutgeo)
n_Γ = get_normal_vector(Γ)
dΓ = Measure(Γ,degree)

Measure()

### Weak form

We take $\beta_K$ constant in $\Omega$

$$
\begin{aligned}
a(u,v) &\doteq \int_{\Omega} \nabla u \cdot \nabla v \ \mathrm{d}\Omega + \int_{\partial\Omega} \beta uv -  (\nabla u \cdot \boldsymbol{n} ) v - (\nabla v \cdot \boldsymbol{n} ) u \ \mathrm{d}\Omega \\
l(v) &\doteq \int_{\partial\Omega} \beta u_{\rm D}v - (\nabla v \cdot \boldsymbol{n} ) u_{\rm D} \ \mathrm{d}\Omega
\end{aligned}
$$

In [27]:
u(x) = x[1] - x[2] # Solution of the problem
const β = 10.0     # Nitsche coefficient
const h = dp[1]/n  # Mesh size

a(u,v) =
  ∫( ∇(v)⋅∇(u) )dΩ +
  ∫( (β/h)*v*u  - v*(n_Γ⋅∇(u)) - (n_Γ⋅∇(v))*u )dΓ

l(v) = ∫( (β/h)*v*u - (n_Γ⋅∇(v))*u )dΓ

l (generic function with 1 method)

The rest of the pipeline is as a standard FE simulation.

In [28]:
op = AffineFEOperator(a,l,U,V)
uh = solve(op)

e = u - uh

l2(u) = sqrt(sum( ∫( u*u )*dΩ ))
h1(u) = sqrt(sum( ∫( u*u + ∇(u)⋅∇(u) )*dΩ ))

el2 = l2(e)
eh1 = h1(e)
ul2 = l2(uh)
uh1 = h1(uh)

using Test
@test el2/ul2 < 1.e-8
@test eh1/uh1 < 1.e-7

Test Passed

**Tutorial done!**

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*